In [1]:
from pymongo import MongoClient
from datetime import datetime, date, timedelta
from unidecode import unidecode

import pandas as pd
import mysql.connector
import warnings

import numpy as np
import re

# Acessando o banco MongoDB Compass
cliente = MongoClient(
    "ddns.cartor.com.br",
    port=57017,
    username='cartor',
    password='Cartor1212',
    authSource='admin',
    authMechanism='DEFAULT'
)

db = cliente.CVM
collection = 'Empresas_fin6'

In [13]:
warnings.filterwarnings('ignore')
dbmysql = mysql.connector.connect(
    user='cartor',
    password='Cartor1212',
    host='ddns.cartor.com.br',
    port='53306', # 43306
    database='console' # os.getenv('MYSQL_DATABASE'),
    )

def fEncontrar_close(fDf, fStock_name, fData_divulgacao):
    try:
        filtro = (fDf['Stock'] == fStock_name) & (fDf['Date'] == fData_divulgacao)
        close_value = fDf.loc[filtro, 'Close'].values[0]
        return close_value
    except IndexError:
        try:
            filtro = (fDf['Stock'] == fStock_name) & (fDf['Date'] == (fData_divulgacao+timedelta(days=1)))
            close_value = fDf.loc[filtro, 'Close'].values[0]
            return close_value            
        except IndexError:        
            try:
                filtro = (fDf['Stock'] == fStock_name) & (fDf['Date'] == (fData_divulgacao+timedelta(days=2)))
                close_value = fDf.loc[filtro, 'Close'].values[0]
                return close_value
            except IndexError:
                try:
                    filtro = (fDf['Stock'] == fStock_name) & (fDf['Date'] == (fData_divulgacao+timedelta(days=2)))
                    close_value = fDf.loc[filtro, 'Close'].values[0]
                    return close_value
                except IndexError:                                                                
                    return None 
    
def fConverterstr_data(fData_hora_string):
    data_hora = datetime.strptime(fData_hora_string, "%d/%m/%Y %H:%M:%S")
    data = data_hora.date()
    return data    

def fExtrair_hora(fData_hora_str):
    data_hora = datetime.strptime(fData_hora_str, "%d/%m/%Y %H:%M:%S")
    horas = data_hora.hour
    if horas > 17:
        aumentar_dias = 1
    else:
        aumentar_dias = 0
    return aumentar_dias

msql = pd.read_sql_query("SELECT * from b3", dbmysql)
msql = msql.reset_index(drop=True)
msql.head(5)

,Date,Stock,Open,Close,Max,Min,Average,Volume
0,2017-01-02,DI1F24,11.51,11.42,11.54,11.41,NaN,NaN
1,2017-01-03,DI1F24,11.44,11.47,11.51,11.44,NaN,NaN
2,2017-01-04,DI1F24,11.56,11.58,11.58,11.56,NaN,NaN
3,2017-01-05,DI1F24,11.53,11.51,11.55,11.46,NaN,NaN
4,2017-01-06,DI1F24,11.47,11.54,11.54,11.47,NaN,NaN


In [3]:
def finddoc_byequity(fDb, fCollection_name, fEquity): # acha o documento da empresa pelo ticker da bolsa 
    count = 0
    while count != 1:
        collection = fDb[fCollection_name]
        documents = collection.find({'equity': fEquity})
        count = collection.count_documents({'equity': fEquity})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byname(fDb, fCollection_name, fContaining_word, fTrimestre=False):
    count = 0
    while count != 1:
        if fTrimestre:
            collection = fDb[fCollection_name]
            regex = re.compile(fContaining_word+fTrimestre, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})
        else:
            collection = fDb[fCollection_name]
            regex = re.compile(fContaining_word, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})
        if count > 1:
            print(type(documents))
            print('erro aconteceu - checar base')
            for doc in documents:
                print(doc['_id'])
            break
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None
    return documents

def upload_infofin(fData_fin, fDoc): # retorna os trimestres da empresa 
    trimestres_upload = []
    for chave, valor in fData_fin.items():
        if isinstance(chave, str) and chave.startswith('2'):
            trimestres_upload.append(chave)
    trimestres_upload = sorted(trimestres_upload, reverse=True)     
    tipo_balanco, trimestres_upload = balance_type(fDoc, trimestres_upload)
    if tipo_balanco == 'ERROR': exit()
    tipo_balanco2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', tipo_balanco)
    return tipo_balanco, tipo_balanco2, trimestres_upload

def balance_type(fEmpresa, fTrimestre): # seleciona o tipo de balanco existente
    tipo_balanco = 'ERROR'   
    # print('\n','entrei aki','\n')
    for key in fEmpresa[fTrimestre[0]]:
        # print(key)
        key2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', key)       
        # print(key2)
        try:
             pvalor = fEmpresa[fTrimestre[-1]][key][key2+' - Demonstração do Resultado']['3.01']['vl_conta']
            #  print(fTrimestre[-1])
            #  print(pvalor)
             if pvalor != 0: 
                 tipo_balanco = key
                 break
        except KeyError:
            # print('\nCia nao possui DF CONSOLIDADO')
            pass
    novalista_trimestres = []
    if tipo_balanco == 'DFConsolidado':
        for trimestre in fTrimestre:
            # print(fEmpresa[trimestre])
            if len(fEmpresa[trimestre].get(tipo_balanco, {})) != 0:
                    novalista_trimestres.append(trimestre) 
        fTrimestre = novalista_trimestres
    # print(f'\nUtilizando balanço: {tipo_balanco}')        
    return tipo_balanco, fTrimestre

def encontrar_conta(fMydict, fConta):
   f_value = 0 
   for key in fMydict:
      string = unidecode(fMydict[key]['ds_conta'].lower())
      if fConta in string:
         f_value = fMydict[key]['vl_conta']
         break
   return f_value

def transform_Dfp_tri(fDicionario, fConta, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2):
    f_valor = 0    
    total_tri = fQuarter+3
    f_valor = fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
    if dt.month == 12:  
        if total_tri > fQtdtrimestres:
            f_valor /= 4
        else:
            f_tri_1 = fDicionario[fTrimestres[fQuarter+1]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
            f_tri_2 = fDicionario[fTrimestres[fQuarter+2]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
            f_tri_3 = fDicionario[fTrimestres[fQuarter+3]][fTipobalanco][fTipobalanco2+' - Demonstração do Resultado'][fConta]['vl_conta']
            f_valor -= (f_tri_1+f_tri_2+f_tri_3) 
    return f_valor

def depreciacao_FDC_tri2(fDicionario, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2): # deletar após testes
    f_valor = 0    
    f_total_tri = fQuarter+1
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
    if f_total_tri+1 >= fQtdtrimestres:
        try:
            f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
            if f_valor==0:
                try:
                    f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                except KeyError:
                    f_valor = 0   
        except KeyError:
            f_valor = 0
        dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
        if dt.month == 12: f_valor /= 4
        elif dt.month == 9: f_valor /= 3
        elif dt.month == 6: f_valor /= 2
    else:
        if dt.month==3:
            try:
                f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
                if f_valor==0:
                    try:
                        f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                    except KeyError:
                        f_valor = 0   
            except KeyError:
                f_valor = 0
        else: 
            # trimestre atual
            try:
                f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
                if f_valor==0:
                    try:
                        f_valor = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                    except KeyError:
                        f_valor = 0   
            except KeyError:
                f_valor = 0
            # trimestre anterior
            f_valor_ant = 0
            try:
                f_valor_ant = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter+1]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacoes'))
                if f_valor_ant==0:
                    try:
                        f_valor_ant = abs(encontrar_conta(fDicionario[fTrimestres[fQuarter+1]][fTipobalanco][fTipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'], 'depreciacao'))
                    except KeyError:
                        f_valor_ant = 0   
            except KeyError:
                f_valor_ant = 0
            f_valor = f_valor - f_valor_ant
    return f_valor

def obter_valor_depreciacao(fDicionario, fTrimestre, fTipobalanco, fTipobalanco2):
    chave = fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Indireto)'
    try:
        valor = abs(encontrar_conta(fDicionario[fTrimestre][fTipobalanco][chave], 'depreciacoes'))
        if valor == 0:
            valor = abs(encontrar_conta(fDicionario[fTrimestre][fTipobalanco][chave], 'depreciacao'))
    except KeyError:
        valor = 0
    return valor

def depreciacao_FDC_tri(fDicionario, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2):
    f_valor = 0    
    f_total_tri = fQuarter + 1
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')

    if  dt.month == 3:
        f_valor = obter_valor_depreciacao(fDicionario, fTrimestres[fQuarter], fTipobalanco, fTipobalanco2)
    else:
        f_valor_atual = obter_valor_depreciacao(fDicionario, fTrimestres[fQuarter], fTipobalanco, fTipobalanco2)
        f_valor_anterior = obter_valor_depreciacao(fDicionario, fTrimestres[fQuarter + 1], fTipobalanco, fTipobalanco2)
        f_valor = f_valor_atual - f_valor_anterior       

    if f_total_tri + 1 >= fQtdtrimestres:
        divisor = {'12': 4, '9': 3, '6': 2}.get(str(dt.month), 1)
        f_valor /= divisor

    return f_valor

def transform_FDC_tri(fDicionario, fConta, fQuarter, fQtdtrimestres, fTrimestres, fTipobalanco, fTipobalanco2):
    f_total_tri = fQuarter + 1
    dt = datetime.strptime(fTrimestres[fQuarter], '%Y-%m-%d')
    try:
        f_valor = fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Indireto)'][fConta]['vl_conta']
        if f_total_tri > fQtdtrimestres:
            f_valor /= 4 if dt.month == 12 else 3 if dt.month == 9 else 2 if dt.month == 6 else 1
        elif dt.month != 3:
            f_valor -= fDicionario.get(fTrimestres[f_total_tri], {}).get(fTipobalanco, {}).get(fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Indireto)', {}).get(fConta, {}).get('vl_conta', 0)
    except KeyError:
        try:
            f_valor = fDicionario[fTrimestres[fQuarter]][fTipobalanco][fTipobalanco2 + ' - Demonstração do Fluxo de Caixa (Método Direto)'][fConta]['vl_conta']
        except KeyError:
            f_valor = 0
    return f_valor

In [4]:
stock = 'RADL3'

In [5]:
document = finddoc_byequity(db, collection, stock)
doc_stock = document.next() # dicionario referente a stock pick
id_empresa = doc_stock['_id']; print(id_empresa)
tipo_balanco1, tipo_balanco2, trimestres_lista = upload_infofin(doc_stock, doc_stock) 

RAIA DROGASIL S.A.


In [6]:
ebit = []
for i in range(len(trimestres_lista)):
    ebit.append(transform_Dfp_tri(doc_stock, '3.05', i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2))
ebit = np.array(ebit)

print(stock)

# calcular 3t, 2t, 1t variacao
variacao_3t = round(ebit[0]/ebit[0+4], 2)
variacao_2t = round(ebit[1]/ebit[1+4], 2) 
variacao_1t = round(ebit[2]/ebit[2+4], 2) 
variacao_21t = round(variacao_2t - variacao_1t, 2)
variacao_32t = round(variacao_3t - variacao_2t, 2)
print(f'Trimestre {trimestres_lista[0]} com variação de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação de {variacao_1t}')
print(f'Variacao do 2 para o 1 = {variacao_21t}')
print(f'Variacao do 3 para o 2 = {variacao_32t}')

# calcular o ebit ltm
variacao_3t = round(np.sum(ebit[0:4])/np.sum(ebit[4:4+4]), 2)
variacao_2t = round(np.sum(ebit[1:1+4])/np.sum(ebit[5:5+4]), 2)
variacao_1t = round(np.sum(ebit[2:2+4])/np.sum(ebit[6:6+4]), 2)
variacao_21ltm = round(variacao_2t - variacao_1t, 2)
variacao_32ltm = round(variacao_3t - variacao_2t, 2)

print(f'Trimestre {trimestres_lista[0]} com variação LTM de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação LTM de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação LTM de {variacao_1t}')
print(f'Variacao do 2 para o 1 ltm = {variacao_21ltm}')
print(f'Variacao do 3 para o 2 ltm = {variacao_32ltm}')

print(f'Total de Ebits {len(ebit)}')
print(f'Total de Trimestres {len(trimestres_lista)}')


RADL3
Trimestre 2023-09-30 com variação de 1.15
Trimestre 2023-06-30 com variação de 1.0
Trimestre 2023-03-31 com variação de 1.57
Variacao do 2 para o 1 = -0.57
Variacao do 3 para o 2 = 0.15
Trimestre 2023-09-30 com variação LTM de 1.23
Trimestre 2023-06-30 com variação LTM de 1.3
Trimestre 2023-03-31 com variação LTM de 1.51
Variacao do 2 para o 1 ltm = -0.21
Variacao do 3 para o 2 ltm = -0.07
Total de Ebits 23
Total de Trimestres 23


In [7]:
ebitda = []
for i in range(len(trimestres_lista)):
    ebit_value = transform_Dfp_tri(doc_stock, '3.05', i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2) 
    depreciation_value = depreciacao_FDC_tri(doc_stock, i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2) 
    ebitda.append(ebit_value+depreciation_value)
ebitda = np.array(ebitda)

print(stock)

# calcular 3t, 2t, 1t variacao
variacao_3t = round(ebitda[0]/ebitda[0+4], 2)
variacao_2t = round(ebitda[1]/ebitda[1+4], 2) 
variacao_1t = round(ebitda[2]/ebitda[2+4], 2) 
variacao_21t = round(variacao_2t - variacao_1t, 2)
variacao_32t = round(variacao_3t - variacao_2t, 2)
print(f'Trimestre {trimestres_lista[0]} com variação de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação de {variacao_1t}')
print(f'Variacao do 2 para o 1 = {variacao_21t}')
print(f'Variacao do 3 para o 2 = {variacao_32t}')

# calcular o ebit ltm
variacao_3t = round(np.sum(ebitda[0:4])/np.sum(ebitda[4:4+4]), 2)
variacao_2t = round(np.sum(ebitda[1:1+4])/np.sum(ebitda[5:5+4]), 2)
variacao_1t = round(np.sum(ebitda[2:2+4])/np.sum(ebitda[6:6+4]), 2)
variacao_21ltm = round(variacao_2t - variacao_1t, 2)
variacao_32ltm = round(variacao_3t - variacao_2t, 2)

print(f'Trimestre {trimestres_lista[0]} com variação LTM de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação LTM de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação LTM de {variacao_1t}')
print(f'Variacao do 2 para o 1 ltm = {variacao_21ltm}')
print(f'Variacao do 3 para o 2 ltm = {variacao_32ltm}')

print(f'Total de Ebitda {len(ebitda)}')
print(f'Total de Trimestres {len(trimestres_lista)}')

RADL3
Trimestre 2023-09-30 com variação de 1.17
Trimestre 2023-06-30 com variação de 1.05
Trimestre 2023-03-31 com variação de 1.31
Variacao do 2 para o 1 = -0.26
Variacao do 3 para o 2 = 0.12
Trimestre 2023-09-30 com variação LTM de 1.19
Trimestre 2023-06-30 com variação LTM de 1.21
Trimestre 2023-03-31 com variação LTM de 1.31
Variacao do 2 para o 1 ltm = -0.1
Variacao do 3 para o 2 ltm = -0.02
Total de Ebitda 23
Total de Trimestres 23


In [8]:
fluxocaixa_operacional = []
for i in range(len(trimestres_lista)):
    fluxocaixa_value = transform_FDC_tri(doc_stock, '6.01.01', i, len(trimestres_lista), trimestres_lista, tipo_balanco1, tipo_balanco2)
    fluxocaixa_operacional.append(fluxocaixa_value)
fluxocaixa_operacional = np.array(fluxocaixa_operacional)

print(stock)

# calcular 3t, 2t, 1t variacao
variacao_3t = round(fluxocaixa_operacional[0]/fluxocaixa_operacional[0+4], 2)
variacao_2t = round(fluxocaixa_operacional[1]/fluxocaixa_operacional[1+4], 2) 
variacao_1t = round(fluxocaixa_operacional[2]/fluxocaixa_operacional[2+4], 2) 
variacao_21t = round(variacao_2t - variacao_1t, 2)
variacao_32t = round(variacao_3t - variacao_2t, 2)
print(f'Trimestre {trimestres_lista[0]} com variação de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação de {variacao_1t}')
print(f'Variacao do 2 para o 1 = {variacao_21t}')
print(f'Variacao do 3 para o 2 = {variacao_32t}')

# calcular o ebit ltm
variacao_3t = round(np.sum(fluxocaixa_operacional[0:4])/np.sum(fluxocaixa_operacional[4:4+4]), 2)
variacao_2t = round(np.sum(fluxocaixa_operacional[1:1+4])/np.sum(fluxocaixa_operacional[5:5+4]), 2)
variacao_1t = round(np.sum(fluxocaixa_operacional[2:2+4])/np.sum(fluxocaixa_operacional[6:6+4]), 2)
variacao_21ltm = round(variacao_2t - variacao_1t, 2)
variacao_32ltm = round(variacao_3t - variacao_2t, 2)

print(f'Trimestre {trimestres_lista[0]} com variação LTM de {variacao_3t}')
print(f'Trimestre {trimestres_lista[1]} com variação LTM de {variacao_2t}')
print(f'Trimestre {trimestres_lista[2]} com variação LTM de {variacao_1t}')
print(f'Variacao do 2 para o 1 ltm = {variacao_21ltm}')
print(f'Variacao do 3 para o 2 ltm = {variacao_32ltm}')

print(f'Total de Fluxo de Caixa Operacional {len(fluxocaixa_operacional)}')
print(f'Total de Trimestres {len(trimestres_lista)}')

RADL3
Trimestre 2023-09-30 com variação de 1.11
Trimestre 2023-06-30 com variação de 1.02
Trimestre 2023-03-31 com variação de 1.29
Variacao do 2 para o 1 = -0.27
Variacao do 3 para o 2 = 0.09
Trimestre 2023-09-30 com variação LTM de 1.16
Trimestre 2023-06-30 com variação LTM de 1.25
Trimestre 2023-03-31 com variação LTM de 1.33
Variacao do 2 para o 1 ltm = -0.08
Variacao do 3 para o 2 ltm = -0.09
Total de Fluxo de Caixa Operacional 23
Total de Trimestres 23


In [29]:
# Encontrar data de divulgacao dos nrs ou definir datas fixas // optar por datas fixas ou variáveis
data_inicio = '10/08/2023 00:00:00'
data_fim = '10/10/2023 00:00:00'

document_inforesultados = finddoc_byname(db, 'divulgacaoinfo_resultados', id_empresa, '_30/06/2023').next()

nome_cia = document_inforesultados['nome_empresa']
hora_envioresultado = document_inforesultados['hora_envio']
dia_posdivulgacao = fConverterstr_data(hora_envioresultado) + timedelta(fExtrair_hora(hora_envioresultado))

preco_inicial = fEncontrar_close(msql, stock, dia_posdivulgacao)
preco_fim = fEncontrar_close(msql, stock, fConverterstr_data(data_fim))
print(f'{stock} _ Preco inicial {preco_inicial} e preco final de {preco_fim}')

preco_inicial = fEncontrar_close(msql, 'BOVA11', dia_posdivulgacao)
preco_fim = fEncontrar_close(msql, 'BOVA11', fConverterstr_data(data_fim))
print(f'BOVA11 _ Preco inicial {preco_inicial} e preco final de {preco_fim}')

print(f'Data Inicial: {fConverterstr_data(data_inicio)}')
print(f'Data Final: {fConverterstr_data(data_fim)}')

RADL3 _ Preco inicial 27.45 e preco final de 28.11
BOVA11 _ Preco inicial None e preco final de None
Data Inicial: 2023-08-10
Data Final: 2023-10-10


In [ ]:
# Stock.inputvalores_primarios(stock_info[i], id_empresa, df2["cod_cvm"], df2["hora_envio"], trimestres_lista[0], fdc)
# shift_dias = extrair_hora(stock_info[i].data_divulg)    
# dia_posdivulgacao = converterstr_data(stock_info[i].data_divulg) + timedelta(shift_dias)
# # preco_inicio = encontrar_close(msql, stock, dia_posdivulgacao)
# preco_inicio = encontrar_close(msql, stock, converterstr_data('10/08/2023 00:00:00'))
# preco_fim = encontrar_close(msql, stock, converterstr_data('10/10/2023 00:00:00'))
# Stock.inputvalores_precos(stock_info[i], preco_inicio, preco_fim)